In [1]:
import os
os.environ['LD_LIBRARY_PATH'] = '/usr/local/lib'

In [2]:
import numpy as np
from numba import jit, vectorize
import time

In [3]:
%load_ext Cython

In [4]:
def brutus_optimize_fit(data, tot_var, models, rvecs, drvecs, av, rv, mag_coeffs,
                  avlim=(0., 20.), av_gauss=(0., 1e6),
                  rvlim=(1., 8.), rv_gauss=(3.32, 0.18),
                  resid=None, tol=0.05, init_thresh=5e-3, stepsize=1.,
                  mags=None, mags_var=None):
    """
    Optimize the distance and reddening between the models and the data using
    the gradient.

    Parameters
    ----------
    data : `~numpy.ndarray` of shape `(Nfilt)`
        Observed data values.

    tot_var : `~numpy.ndarray` of shape `(Nmodel, Nfilt)`
        Associated (Normal) errors on the observed values compared to the
        models.

    models : `~numpy.ndarray` of shape `(Nmodel, Nfilt)`
        Model predictions.

    rvecs : `~numpy.ndarray` of shape `(Nmodel, Nfilt)`
        Associated model reddening vectors.

    drvecs : `~numpy.ndarray` of shape `(Nmodel, Nfilt)`
        Associated differential model reddening vectors.

    av : `~numpy.ndarray` of shape `(Nmodel,)`
        Av values of the models.

    rv : `~numpy.ndarray` of shape `(Nmodel,)`
        Rv values of the models.

    mag_coeffs : `~numpy.ndarray` of shape `(Nmodel, Nfilt, 3)`
        Magnitude coefficients used to compute reddened photometry for a given
        model.

    avlim : 2-tuple, optional
        The lower and upper bound where the reddened photometry is reliable.
        Default is `(0., 20.)`.

    av_gauss : 2-tuple, optional
        The mean and standard deviation of the Gaussian prior that is placed
        on A(V). The default is `(0., 1e6)`, which is designed to be
        essentially flat over `avlim`.

    rvlim : 2-tuple, optional
        The lower and upper bound where the reddening vector shape changes
        are reliable. Default is `(1., 8.)`.

    rv_gauss : 2-tuple, optional
        The mean and standard deviation of the Gaussian prior that is placed
        on R(V). The default is `(3.32, 0.18)` based on the results from
        Schlafly et al. (2016).

    resid : `~numpy.ndarray` of shape `(Nmodel, Nfilt)`
        Residuals between the data and models.
        If not provided, this will be computed.

    tol : float, optional
        The maximum tolerance in the computed Av and Rv values used to
        determine convergence during the magnitude fits. Default is `0.05`.

    init_thresh : bool, optional
        The weight threshold used to mask out fits after the initial
        magnitude-based fit before transforming the results back to
        flux density (and iterating until convergence). Default is `5e-3`.

    stepsize : float or `~numpy.ndarray`, optional
        The stepsize (in units of the computed gradient). Default is `1.`.

    mags : `~numpy.ndarray` of shape `(Nfilt)`, optional
        Observed data values in magnitudes.

    mags_var : `~numpy.ndarray` of shape `(Nmodel, Nfilt)`, optional
        Associated (Normal) errors on the observed values compared to the
        models in magnitudes.

    Returns
    -------
    models_new : `~numpy.ndarray` of shape `(Nmodel, Nfilt)`
        New model predictions. Always returned in flux densities.

    rvecs_new : `~numpy.ndarray` of shape `(Nmodel, Nfilt)`
        New reddening vectors. Always returned in flux densities.

    drvecs_new : `~numpy.ndarray` of shape `(Nmodel, Nfilt)`
        New differential reddening vectors. Always returned in flux densities.

    scale : `~numpy.ndarray` of shape `(Nmodel)`, optional
        The best-fit scale factor.

    Av : `~numpy.ndarray` of shape `(Nmodel)`, optional
        The best-fit reddening.

    Rv : `~numpy.ndarray` of shape `(Nmodel)`, optional
        The best-fit reddening shapes.

    icov_sar : `~numpy.ndarray` of shape `(Nmodel, 3, 3)`, optional
        The precision (inverse covariance) matrices expanded around
        `(s_ML, Av_ML, Rv_ML)`.

    resid : `~numpy.ndarray` of shape `(Nmodel, Nfilt)`
        Residuals between the data and models.

    """

    # Compute residuals.
    if resid is None:
        if mags is not None and mags_var is not None:
            resid = mags - models
        else:
            resid = data - models

    Av_mean, Av_std = av_gauss
    Rv_mean, Rv_std = rv_gauss

    if mags is not None and mags_var is not None:
        # If magnitudes are provided, we can solve the linear system
        # explicitly for `(s_ML, Av_ML, r_ML=Av_ML*Rv_ML)`. We opt to
        # solve for Av and Rv in turn to so we can impose priors and bounds
        # on both quantities.

        # Compute constants.
        s_den = np.sum(1. / mags_var, axis=1)
        rp_den = np.sum(np.square(drvecs) / mags_var, axis=1)
        srp_mix = np.sum(drvecs / mags_var, axis=1)

        err = 1e300
        while err > tol:
            # Solve for Av.
            # Derive partial derivatives.
            a_den = np.sum(np.square(rvecs) / mags_var, axis=1)
            sa_mix = np.sum(rvecs / mags_var, axis=1)
            # Compute residual terms.
            resid_s = np.sum(resid / mags_var, axis=1)
            resid_a = np.sum(resid * rvecs / mags_var, axis=1)
            # Add in Gaussian Av prior.
            resid_a += (Av_mean - av) / Av_std**2
            a_den += 1. / Av_std**2
            # Compute determinants (normalization terms).
            sa_idet = 1. / (s_den * a_den - sa_mix**2)
            # Compute ML solution for Delta_Av.
            dav = sa_idet * (s_den * resid_a - sa_mix * resid_s)

            # Prevent Av from sliding off the provided bounds.
            dav_low, dav_high = avlim[0] - av, avlim[1] - av
            lsel, hsel = dav < dav_low, dav > dav_high
            dav[lsel] = dav_low[lsel]
            dav[hsel] = dav_high[hsel]

            # Increment to new Av.
            av += dav
            # Update residuals.
            resid -= dav[:, None] * rvecs  # update residuals

            # Solve for Rv.
            # Derive partial derivatives.
            r_den = rp_den * av**2
            sr_mix = srp_mix * av
            # Compute residual terms.
            resid_s = np.sum(resid / mags_var, axis=1)
            resid_r = np.sum(resid * drvecs / mags_var, axis=1) * av
            # Add in Gaussian Rv prior.
            resid_r += (Rv_mean - rv) / Rv_std**2
            r_den += 1. / Rv_std**2
            # Compute determinants (normalization terms).
            sr_idet = 1. / (s_den * r_den - sr_mix**2)
            # Compute ML solution for Delta_Rv.
            drv = sr_idet * (s_den * resid_r - sr_mix * resid_s)

            # Prevent Rv from sliding off the provided bounds.
            drv_low, drv_high = rvlim[0] - rv, rvlim[1] - rv
            lsel, hsel = drv < drv_low, drv > drv_high
            drv[lsel] = drv_low[lsel]
            drv[hsel] = drv_high[hsel]

            # Increment to new Rv.
            rv += drv
            # Update residuals.
            resid -= (av * drv)[:, None] * drvecs
            # Update reddening vector.
            rvecs += drv[:, None] * drvecs

            # Compute error based on best-fitting objects.
            chi2 = np.sum(np.square(resid) / mags_var, axis=1)
            logwt = -0.5 * chi2
            init_sel = np.where(logwt > np.max(logwt) + np.log(init_thresh))[0]
            err = np.max([np.abs(dav[init_sel]), np.abs(drv[init_sel])])
    else:
        # If our data is in flux densities, we can solve the linear system
        # implicitly for `(s_ML, Av_ML, Rv_ML)`. However, the solution
        # is not necessarily as numerically stable as one might hope
        # due to the nature of our Taylor expansion in flux.
        # Instead, it is easier to iterate in `(dAv, dRv)` from
        # a good guess for `(s_ML, Av_ML, Rv_ML)`. We opt to solve both
        # independently at fixed `(Av, Rv)` to avoid recomputing models.

        # Derive ML Delta_Av (`dav`) between data and models.
        a_num = np.sum(rvecs * resid / tot_var, axis=1)
        a_den = np.sum(np.square(rvecs) / tot_var, axis=1)
        a_num += (Av_mean - av) / Av_std**2  # add Av gaussian prior
        a_den += 1. / Av_std**2  # add Av gaussian prior
        dav = a_num / a_den
        # Adjust dAv based on the provided stepsize.
        dav *= stepsize

        # Derive ML Delta_Rv (`drv`) between data and models.
        r_num = np.sum(drvecs * resid / tot_var, axis=1)
        r_den = np.sum(np.square(drvecs) / tot_var, axis=1)
        r_num += (Rv_mean - rv) / Rv_std**2  # add Rv gaussian prior
        r_den += 1. / Rv_std**2  # add Rv gaussian prior
        drv = r_num / r_den
        # Adjust dRv based on the provided stepsize.
        drv *= stepsize

        # Prevent Av from sliding off the provided bounds.
        dav_low, dav_high = avlim[0] - av, avlim[1] - av
        lsel, hsel = dav < dav_low, dav > dav_high
        dav[lsel] = dav_low[lsel]
        dav[hsel] = dav_high[hsel]
        # Increment to new Av.
        av += dav

        # Prevent Rv from sliding off the provided bounds.
        drv_low, drv_high = rvlim[0] - rv, rvlim[1] - rv
        lsel, hsel = drv < drv_low, drv > drv_high
        drv[lsel] = drv_low[lsel]
        drv[hsel] = drv_high[hsel]
        # Increment to new Rv.
        rv += drv

    # Recompute models with new Rv.
    models, rvecs, drvecs = brutus_get_seds(mag_coeffs, av=av, rv=rv,
                                     return_flux=True,
                                     return_rvec=True, return_drvec=True)

    # Derive scale-factors (`scale`) between data and models.
    s_num = np.sum(models * data[None, :] / tot_var, axis=1)
    s_den = np.sum(np.square(models) / tot_var, axis=1)
    scale = s_num / s_den  # ML scalefactor
    scale[scale <= 1e-20] = 1e-20  # must be non-negative

    # Compute reddening effect.
    models_int = 10**(-0.4 * mag_coeffs[:, :, 0])
    reddening = models - models_int

    # Rescale models.
    models *= scale[:, None]

    # Compute residuals.
    resid = data - models

    # Derive scale cross-terms.
    sr_mix = np.sum(drvecs * (resid - models) / tot_var, axis=1)
    sa_mix = np.sum(rvecs * (resid - models) / tot_var, axis=1)

    # Rescale reddening quantities.
    rvecs *= scale[:, None]
    drvecs *= scale[:, None]
    reddening *= scale[:, None]

    # Deriving reddening (cross-)terms.
    ar_mix = np.sum(drvecs * (resid - reddening) / tot_var, axis=1)
    a_den = np.sum(np.square(rvecs) / tot_var, axis=1)
    r_den = np.sum(np.square(drvecs) / tot_var, axis=1)
    r_den += 1. / Rv_std**2  # add Rv gaussian prior

    # Construct precision matrices (inverse covariances).
    icov_sar = np.zeros((len(models), 3, 3))
    icov_sar[:, 0, 0] = s_den  # scale
    icov_sar[:, 1, 1] = a_den  # Av
    icov_sar[:, 2, 2] = r_den  # Rv
    icov_sar[:, 0, 1] = sa_mix  # scale-Av cross-term
    icov_sar[:, 1, 0] = sa_mix  # scale-Av cross-term
    icov_sar[:, 0, 2] = sr_mix  # scale-Rv cross-term
    icov_sar[:, 2, 0] = sr_mix  # scale-Rv cross-term
    icov_sar[:, 1, 2] = ar_mix  # Av-Rv cross-term
    icov_sar[:, 2, 1] = ar_mix  # Av-Rv cross-term

    return models, rvecs, drvecs, scale, av, rv, icov_sar, resid

In [5]:
def brutus_get_seds(mag_coeffs, av=None, rv=None, return_flux=False,
             return_rvec=False, return_drvec=False):
    """
    Compute reddened SEDs from the provided magnitude coefficients.

    Parameters
    ----------
    mag_coeffs : `~numpy.ndarray` of shape `(Nmodels, Nbands, 3)`
        Array of `(mag, R, dR/dRv)` coefficients used to generate
        reddened photometry in all bands. The first coefficient is the
        unreddened photometry, the second is the A(V) reddening vector for
        R(V)=0, and the third is the change in the reddening vector
        as a function of R(V).

    av : float or `~numpy.ndarray` of shape `(Nmodels)`, optional
        Array of A(V) dust attenuation values.
        If not provided, defaults to `av=0.`.

    rv : float or `~numpy.ndarray` of shape `(Nmodels)`, optional
        Array of R(V) dust attenuation curve "shape" values.
        If not provided, defaults to `rv=3.3`.

    return_flux : bool, optional
        Whether to return SEDs as flux densities instead of magnitudes.
        Default is `False`.

    return_rvec : bool, optional
        Whether to return the reddening vectors at the provided
        `av` and `rv`. Default is `False`.

    return_drvec : bool, optional
        Whether to return the differential reddening vectors at the provided
        `av` and `rv`. Default is `False`.

    Returns
    -------
    seds : `~numpy.ndarray` of shape `(Nmodels, Nbands)`
        Reddened SEDs.

    rvecs : `~numpy.ndarray` of shape `(Nmodels, Nbands)`, optional
        Reddening vectors.

    drvecs : `~numpy.ndarray` of shape `(Nmodels, Nbands)`, optional
        Differential reddening vectors.

    """

    Nmodels, Nbands, Ncoef = mag_coeffs.shape
    if av is None:
        av = np.zeros(Nmodels)
    # elif isinstance(av, (int, float)):
    else:
        av = np.full(Nmodels, av)
    if rv is None:
        rv = np.full(Nmodels, 3.3)
    # elif isinstance(rv, (int, float)):
    else:
        rv = np.full(Nmodels, rv)

    # Turn provided Av values into polynomial features.
    mags = mag_coeffs[:, :, 0]
    r0 = mag_coeffs[:, :, 1]
    dr = mag_coeffs[:, :, 2]

    # Compute SEDs.
    drvecs = np.array(dr)
    rvecs = r0 + rv[:, None] * drvecs
    seds = mags + av[:, None] * rvecs

    # Convert to flux.
    if return_flux:
        seds = 10**(-0.4 * seds)
        if return_rvec:
            rvecs *= -0.4 * np.log(10.) * seds
        if return_drvec:
            drvecs *= -0.4 * np.log(10.) * seds

    if return_rvec and return_drvec:
        return seds, rvecs, drvecs
    elif return_rvec:
        return seds, rvecs
    elif return_drvec:
        return seds, drvecs
    else:
        return seds

In [6]:
flux, tot_var, models, rvecs, drvecs, av_init, rv_init, mcoeffs,\
  mtol, init_thresh, mags, mags_var, avlim, av_gauss,\
   rvlim, rv_gauss = np.load('optimize_fit.npy', allow_pickle=True)

In [7]:
import cProfile

In [8]:
cProfile.run("brutus_optimize_fit(flux, tot_var, models, rvecs, drvecs, av_init, rv_init, mcoeffs,\
              tol=mtol, init_thresh=init_thresh,\
              resid=None, mags=mags, mags_var=mags_var,\
              avlim=avlim, av_gauss=av_gauss,\
              rvlim=rvlim, rv_gauss=rv_gauss)", "profile_optimize_fit.dat")

In [9]:
import pstats
from pstats import SortKey

In [10]:
p = pstats.Stats('profile_optimize_fit.dat')
# p.strip_dirs().sort_stats(-1).print_stats()

In [11]:
p.sort_stats(SortKey.TIME).print_stats(30)

Sat May 16 14:41:45 2020    profile_optimize_fit.dat

         276 function calls in 0.927 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.549    0.549    0.902    0.902 <ipython-input-4-1553e451d585>:1(brutus_optimize_fit)
       28    0.209    0.007    0.209    0.007 {method 'reduce' of 'numpy.ufunc' objects}
        1    0.133    0.133    0.144    0.144 <ipython-input-5-04531aa9500b>:1(brutus_get_seds)
        1    0.025    0.025    0.927    0.927 <string>:1(<module>)
        3    0.010    0.003    0.010    0.003 {built-in method numpy.array}
       32    0.002    0.000    0.211    0.007 {built-in method numpy.core._multiarray_umath.implement_array_function}
       28    0.000    0.000    0.209    0.007 /usr/local/lib/python3.7/site-packages/numpy/core/fromnumeric.py:73(_wrapreduction)
       24    0.000    0.000    0.208    0.009 /usr/local/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2092(sum)
 

# Optimized Version

In [28]:
@jit(nopython=True)
def get_seds(mag_coeffs, av=None, rv=None, return_flux=False,
             return_rvec=False, return_drvec=False):
    """
    Compute reddened SEDs from the provided magnitude coefficients.

    Parameters
    ----------
    mag_coeffs : `~numpy.ndarray` of shape `(Nmodels, Nbands, 3)`
        Array of `(mag, R, dR/dRv)` coefficients used to generate
        reddened photometry in all bands. The first coefficient is the
        unreddened photometry, the second is the A(V) reddening vector for
        R(V)=0, and the third is the change in the reddening vector
        as a function of R(V).

    av : float or `~numpy.ndarray` of shape `(Nmodels)`, optional
        Array of A(V) dust attenuation values.
        If not provided, defaults to `av=0.`.

    rv : float or `~numpy.ndarray` of shape `(Nmodels)`, optional
        Array of R(V) dust attenuation curve "shape" values.
        If not provided, defaults to `rv=3.3`.

    return_flux : bool, optional
        Whether to return SEDs as flux densities instead of magnitudes.
        Default is `False`.

    return_rvec : bool, optional
        Whether to return the reddening vectors at the provided
        `av` and `rv`. Default is `False`.

    return_drvec : bool, optional
        Whether to return the differential reddening vectors at the provided
        `av` and `rv`. Default is `False`.

    Returns
    -------
    seds : `~numpy.ndarray` of shape `(Nmodels, Nbands)`
        Reddened SEDs.

    rvecs : `~numpy.ndarray` of shape `(Nmodels, Nbands)`, optional
        Reddening vectors.

    drvecs : `~numpy.ndarray` of shape `(Nmodels, Nbands)`, optional
        Differential reddening vectors.

    """

    Nmodels, Nbands, Ncoef = mag_coeffs.shape
    if av is None:
        av = np.zeros(Nmodels)
    # elif isinstance(av, (int, float)):
    else:
        av = np.full((Nmodels,), av)
    if rv is None:
        rv = np.full((Nmodels,), 3.3)
    # elif isinstance(rv, (int, float)):
    else:
        rv = np.full((Nmodels,), rv)

    # Turn provided Av values into polynomial features.
    mags = mag_coeffs[:, :, 0]
    r0 = mag_coeffs[:, :, 1]
    dr = mag_coeffs[:, :, 2]

    # Compute SEDs.
    drvecs = np.array(dr)
#     rvecs = r0 + rv[:, None] * drvecs
    rvecs  = r0 + np.reshape(rv, (Nmodels, 1)) * drvecs
#     seds = mags + av[:, None] * rvecs
    seds   = mags + np.reshape(av, (Nmodels, 1)) * rvecs

    # Convert to flux.
    if return_flux:
        seds = 10**(-0.4 * seds)
        if return_rvec:
            rvecs *= -0.4 * np.log(10.) * seds
        if return_drvec:
            drvecs *= -0.4 * np.log(10.) * seds

    if return_rvec and return_drvec:
        return seds, rvecs, drvecs
    elif return_rvec:
        return seds, rvecs
    elif return_drvec:
        return seds, drvecs
    else:
        return seds

In [29]:
@jit(nopython=True, cache=True, fastmath=True)
def _get_Av_mags(mags, mags_var, rvecs, drvecs, av, rv, 
                 avlim, av_gauss, rvlim, rv_gauss, resid, tol, init_thresh,
                  Nmodel, Nfilt):
#     Nmodel, Nfilt = np.shape(rvecs)
    
    Av_mean, Av_std = av_gauss
    Rv_mean, Rv_std = rv_gauss
    
    # Compute constants.
    s_den = np.sum(1. / mags_var, axis=1)
    rp_den = np.sum(np.square(drvecs) / mags_var, axis=1)
    srp_mix = np.sum(drvecs / mags_var, axis=1)

    err = 1e300
    while err > tol:
        # Solve for Av.
        # Derive partial derivatives.
        a_den = np.sum(np.square(rvecs) / mags_var, axis=1)
        sa_mix = np.sum(rvecs / mags_var, axis=1)
        # Compute residual terms.
        resid_s = np.sum(resid / mags_var, axis=1)
        resid_a = np.sum(resid * rvecs / mags_var, axis=1)
        # Add in Gaussian Av prior.
        resid_a = resid_a + (Av_mean - av) / Av_std**2
        a_den = a_den + 1. / Av_std**2
        # Compute determinants (normalization terms).
        sa_idet = 1. / (s_den * a_den - sa_mix**2)
        # Compute ML solution for Delta_Av.
        dav = sa_idet * (s_den * resid_a - sa_mix * resid_s)

        # Prevent Av from sliding off the provided bounds.
        dav_low, dav_high = avlim[0] - av, avlim[1] - av
        lsel, hsel = dav < dav_low, dav > dav_high
        dav[lsel] = dav_low[lsel]
        dav[hsel] = dav_high[hsel]

        # Increment to new Av.
        av = av + dav
        # Update residuals.
#         resid -= dav[:, None] * rvecs  # update residuals
        resid = resid - np.reshape(dav, (Nmodel, 1)) * rvecs

        # Solve for Rv.
        # Derive partial derivatives.
        r_den = rp_den * av**2
        sr_mix = srp_mix * av
        # Compute residual terms.
        resid_s = np.sum(resid / mags_var, axis=1)
        resid_r = np.sum(resid * drvecs / mags_var, axis=1) * av
        # Add in Gaussian Rv prior.
        resid_r = resid_r + (Rv_mean - rv) / Rv_std**2
        r_den = r_den + 1. / Rv_std**2
        # Compute determinants (normalization terms).
        sr_idet = 1. / (s_den * r_den - sr_mix**2)
        # Compute ML solution for Delta_Rv.
        drv = sr_idet * (s_den * resid_r - sr_mix * resid_s)

        # Prevent Rv from sliding off the provided bounds.
        drv_low, drv_high = rvlim[0] - rv, rvlim[1] - rv
        lsel, hsel = drv < drv_low, drv > drv_high
        drv[lsel] = drv_low[lsel]
        drv[hsel] = drv_high[hsel]

        # Increment to new Rv.
        rv = rv + drv
        # Update residuals.
#         resid -= (av * drv)[:, None] * drvecs
        resid = resid - np.reshape(av*drv, (Nmodel, 1)) * drvecs
        # Update reddening vector.
#         rvecs += drv[:, None] * drvecs
        rvecs = rvecs + np.reshape(drv, (Nmodel, 1)) * drvecs

        # Compute error based on best-fitting objects.
        chi2 = np.sum(np.square(resid) / mags_var, axis=1)
        logwt = -0.5 * chi2
        init_sel = np.where(logwt > np.max(logwt) + np.log(init_thresh))[0]
        err = np.max(np.concatenate((np.abs(dav[init_sel]), np.abs(drv[init_sel]))))
  
    return rv, rvecs, resid

In [30]:
def _optimize_fit(data, tot_var, models, rvecs, drvecs, av, rv, mag_coeffs,
                  avlim=(0., 20.), av_gauss=(0., 1e6),
                  rvlim=(1., 8.), rv_gauss=(3.32, 0.18),
                  resid=None, tol=0.05, init_thresh=5e-3, stepsize=1.,
                  mags=None, mags_var=None):
    """
    Optimize the distance and reddening between the models and the data using
    the gradient.

    Parameters
    ----------
    data : `~numpy.ndarray` of shape `(Nfilt)`
        Observed data values.

    tot_var : `~numpy.ndarray` of shape `(Nmodel, Nfilt)`
        Associated (Normal) errors on the observed values compared to the
        models.

    models : `~numpy.ndarray` of shape `(Nmodel, Nfilt)`
        Model predictions.

    rvecs : `~numpy.ndarray` of shape `(Nmodel, Nfilt)`
        Associated model reddening vectors.

    drvecs : `~numpy.ndarray` of shape `(Nmodel, Nfilt)`
        Associated differential model reddening vectors.

    av : `~numpy.ndarray` of shape `(Nmodel,)`
        Av values of the models.

    rv : `~numpy.ndarray` of shape `(Nmodel,)`
        Rv values of the models.

    mag_coeffs : `~numpy.ndarray` of shape `(Nmodel, Nfilt, 3)`
        Magnitude coefficients used to compute reddened photometry for a given
        model.

    avlim : 2-tuple, optional
        The lower and upper bound where the reddened photometry is reliable.
        Default is `(0., 20.)`.

    av_gauss : 2-tuple, optional
        The mean and standard deviation of the Gaussian prior that is placed
        on A(V). The default is `(0., 1e6)`, which is designed to be
        essentially flat over `avlim`.

    rvlim : 2-tuple, optional
        The lower and upper bound where the reddening vector shape changes
        are reliable. Default is `(1., 8.)`.

    rv_gauss : 2-tuple, optional
        The mean and standard deviation of the Gaussian prior that is placed
        on R(V). The default is `(3.32, 0.18)` based on the results from
        Schlafly et al. (2016).

    resid : `~numpy.ndarray` of shape `(Nmodel, Nfilt)`
        Residuals between the data and models.
        If not provided, this will be computed.

    tol : float, optional
        The maximum tolerance in the computed Av and Rv values used to
        determine convergence during the magnitude fits. Default is `0.05`.

    init_thresh : bool, optional
        The weight threshold used to mask out fits after the initial
        magnitude-based fit before transforming the results back to
        flux density (and iterating until convergence). Default is `5e-3`.

    stepsize : float or `~numpy.ndarray`, optional
        The stepsize (in units of the computed gradient). Default is `1.`.

    mags : `~numpy.ndarray` of shape `(Nfilt)`, optional
        Observed data values in magnitudes.

    mags_var : `~numpy.ndarray` of shape `(Nmodel, Nfilt)`, optional
        Associated (Normal) errors on the observed values compared to the
        models in magnitudes.

    Returns
    -------
    models_new : `~numpy.ndarray` of shape `(Nmodel, Nfilt)`
        New model predictions. Always returned in flux densities.

    rvecs_new : `~numpy.ndarray` of shape `(Nmodel, Nfilt)`
        New reddening vectors. Always returned in flux densities.

    drvecs_new : `~numpy.ndarray` of shape `(Nmodel, Nfilt)`
        New differential reddening vectors. Always returned in flux densities.

    scale : `~numpy.ndarray` of shape `(Nmodel)`, optional
        The best-fit scale factor.

    Av : `~numpy.ndarray` of shape `(Nmodel)`, optional
        The best-fit reddening.

    Rv : `~numpy.ndarray` of shape `(Nmodel)`, optional
        The best-fit reddening shapes.

    icov_sar : `~numpy.ndarray` of shape `(Nmodel, 3, 3)`, optional
        The precision (inverse covariance) matrices expanded around
        `(s_ML, Av_ML, Rv_ML)`.

    resid : `~numpy.ndarray` of shape `(Nmodel, Nfilt)`
        Residuals between the data and models.

    """
    
    Nmodel, Nfilt = np.shape(models)
    print('Nmodel=', Nmodel, 'Nfilt=', Nfilt)

    # Compute residuals.
    if resid is None:
        if mags is not None and mags_var is not None:
            resid = mags - models
        else:
            resid = data - models

    Av_mean, Av_std = av_gauss
    Rv_mean, Rv_std = rv_gauss
    
    t0 = time.time()
    if mags is not None and mags_var is not None:
        # If magnitudes are provided, we can solve the linear system
        # explicitly for `(s_ML, Av_ML, r_ML=Av_ML*Rv_ML)`.
        rv, rvecs, resid = _get_Av_mags(mags, mags_var, rvecs, drvecs, av, rv, 
                                        avlim, av_gauss, rvlim, rv_gauss, resid, tol, init_thresh,
                                        Nmodel, Nfilt)
        
    else:
        # If our data is in flux densities, we can solve the linear system
        # implicitly for `(s_ML, Av_ML, Rv_ML)`. However, the solution
        # is not necessarily as numerically stable as one might hope
        # due to the nature of our Taylor expansion in flux.
        # Instead, it is easier to iterate in `(dAv, dRv)` from
        # a good guess for `(s_ML, Av_ML, Rv_ML)`. We opt to solve both
        # independently at fixed `(Av, Rv)` to avoid recomputing models.

        # Derive ML Delta_Av (`dav`) between data and models.
        a_num = np.sum(rvecs * resid / tot_var, axis=1)
        a_den = np.sum(np.square(rvecs) / tot_var, axis=1)
        a_num += (Av_mean - av) / Av_std**2  # add Av gaussian prior
        a_den += 1. / Av_std**2  # add Av gaussian prior
        dav = a_num / a_den
        # Adjust dAv based on the provided stepsize.
        dav *= stepsize

        # Derive ML Delta_Rv (`drv`) between data and models.
        r_num = np.sum(drvecs * resid / tot_var, axis=1)
        r_den = np.sum(np.square(drvecs) / tot_var, axis=1)
        r_num += (Rv_mean - rv) / Rv_std**2  # add Rv gaussian prior
        r_den += 1. / Rv_std**2  # add Rv gaussian prior
        drv = r_num / r_den
        # Adjust dRv based on the provided stepsize.
        drv *= stepsize

        # Prevent Av from sliding off the provided bounds.
        dav_low, dav_high = avlim[0] - av, avlim[1] - av
        lsel, hsel = dav < dav_low, dav > dav_high
        dav[lsel] = dav_low[lsel]
        dav[hsel] = dav_high[hsel]
        # Increment to new Av.
        av += dav

        # Prevent Rv from sliding off the provided bounds.
        drv_low, drv_high = rvlim[0] - rv, rvlim[1] - rv
        lsel, hsel = drv < drv_low, drv > drv_high
        drv[lsel] = drv_low[lsel]
        drv[hsel] = drv_high[hsel]
        # Increment to new Rv.
        rv += drv

    t1 = time.time()
#     print('time in first part = ', t1-t0, 'tot, mean in while loop=', np.sum(tlist), np.mean(tlist))
    print('time in first part = ', t1-t0)
        
    # Recompute models with new Rv.
    models, rvecs, drvecs = get_seds(mag_coeffs, av=av, rv=rv,
                                     return_flux=True,
                                     return_rvec=True, return_drvec=True)

    # Derive scale-factors (`scale`) between data and models.
    s_num = np.sum(models * np.reshape(data, (1, Nfilt)) / tot_var, axis=1)
    s_den = np.sum(np.square(models) / tot_var, axis=1)
    scale = s_num / s_den  # ML scalefactor
    scale[scale <= 1e-20] = 1e-20  # must be non-negative

    # Compute reddening effect.
    models_int = 10**(-0.4 * mag_coeffs[:, :, 0])
    reddening = models - models_int

    # Rescale models.
#     models *= scale[:, None]
    models = models * np.reshape(scale, (Nmodel, 1))

    # Compute residuals.
    resid = data - models

    # Derive scale cross-terms.
    sr_mix = np.sum(drvecs * (resid - models) / tot_var, axis=1)
    sa_mix = np.sum(rvecs * (resid - models) / tot_var, axis=1)

    # Rescale reddening quantities.
    
    rvecs     = rvecs     * np.reshape(scale, (Nmodel, 1))
    drvecs    = drvecs    * np.reshape(scale, (Nmodel, 1))
    reddening = reddening * np.reshape(scale, (Nmodel, 1))

    # Deriving reddening (cross-)terms.
    ar_mix = np.sum(drvecs * (resid - reddening) / tot_var, axis=1)
    a_den  = np.sum(np.square(rvecs) / tot_var, axis=1)
    r_den  = np.sum(np.square(drvecs) / tot_var, axis=1)
    r_den  = r_den + 1. / Rv_std**2  # add Rv gaussian prior

    # Construct precision matrices (inverse covariances).
    icov_sar = np.zeros((len(models), 3, 3))
    icov_sar[:, 0, 0] = s_den  # scale
    icov_sar[:, 1, 1] = a_den  # Av
    icov_sar[:, 2, 2] = r_den  # Rv
    icov_sar[:, 0, 1] = sa_mix  # scale-Av cross-term
    icov_sar[:, 1, 0] = sa_mix  # scale-Av cross-term
    icov_sar[:, 0, 2] = sr_mix  # scale-Rv cross-term
    icov_sar[:, 2, 0] = sr_mix  # scale-Rv cross-term
    icov_sar[:, 1, 2] = ar_mix  # Av-Rv cross-term
    icov_sar[:, 2, 1] = ar_mix  # Av-Rv cross-term

    return models, rvecs, drvecs, scale, av, rv, icov_sar, resid

In [31]:
%%timeit
ans_opt = _optimize_fit(flux, tot_var, models, rvecs, drvecs, av_init, rv_init, mcoeffs,
              tol=mtol, init_thresh=init_thresh,
              resid=None, mags=mags, mags_var=mags_var,
              avlim=avlim, av_gauss=av_gauss,
              rvlim=rvlim, rv_gauss=rv_gauss);

Nmodel= 745055 Nfilt= 5
time in first part =  8.23096776008606


TypingError: Failed in nopython mode pipeline (step: nopython frontend)
Invalid use of Function(<function full at 0x114937b90>) with argument(s) of type(s): (UniTuple(int64 x 1), array(float64, 1d, C))
 * parameterized
In definition 0:
    TypeError: Buffer dtype cannot be buffer, have dtype: array(float64, 1d, C)
    raised from /usr/local/lib/python3.7/site-packages/numba/types/common.py:53
In definition 1:
    TypeError: Buffer dtype cannot be buffer, have dtype: array(float64, 1d, C)
    raised from /usr/local/lib/python3.7/site-packages/numba/types/common.py:53
This error is usually caused by passing an argument of a type that is unsupported by the named function.
[1] During: resolving callee type: Function(<function full at 0x114937b90>)
[2] During: typing of call at <ipython-input-28-184f596d5c93> (54)


File "<ipython-input-28-184f596d5c93>", line 54:
def get_seds(mag_coeffs, av=None, rv=None, return_flux=False,
    <source elided>
    else:
        av = np.full((Nmodels,), av)

In [130]:
%%timeit
ans_brutus = brutus_optimize_fit(flux, tot_var, models, rvecs, drvecs, av_init, rv_init, mcoeffs,
              tol=mtol, init_thresh=init_thresh,
              resid=None, mags=mags, mags_var=mags_var,
              avlim=avlim, av_gauss=av_gauss,
              rvlim=rvlim, rv_gauss=rv_gauss);

999 ms ± 83.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [50]:
print(ans_opt[0][0])

[1.22729561e-06 3.33888243e-06 7.69763783e-06 1.17094667e-05
 1.26451980e-05]


In [132]:
print(ans_brutus[0][0])

[1.22729561e-06 3.33888243e-06 7.69763783e-06 1.17094667e-05
 1.26451980e-05]


In [149]:
t = np.random.rand(750000)

In [150]:
def sum_numpy(t):
    return np.sum(t)

In [152]:
@jit(nopython=True, cache=True)
def sum_numba(t):
    return np.sum(t)

In [154]:
%%timeit
sum_numpy(t)

155 µs ± 4.13 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [156]:
%%timeit
sum_numba(t)

653 µs ± 18.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
def get_seds(mag_coeffs, av=None, rv=None, return_flux=False,
             return_rvec=False, return_drvec=False):
    """
    Compute reddened SEDs from the provided magnitude coefficients.

    Parameters
    ----------
    mag_coeffs : `~numpy.ndarray` of shape `(Nmodels, Nbands, 3)`
        Array of `(mag, R, dR/dRv)` coefficients used to generate
        reddened photometry in all bands. The first coefficient is the
        unreddened photometry, the second is the A(V) reddening vector for
        R(V)=0, and the third is the change in the reddening vector
        as a function of R(V).

    av : float or `~numpy.ndarray` of shape `(Nmodels)`, optional
        Array of A(V) dust attenuation values.
        If not provided, defaults to `av=0.`.

    rv : float or `~numpy.ndarray` of shape `(Nmodels)`, optional
        Array of R(V) dust attenuation curve "shape" values.
        If not provided, defaults to `rv=3.3`.

    return_flux : bool, optional
        Whether to return SEDs as flux densities instead of magnitudes.
        Default is `False`.

    return_rvec : bool, optional
        Whether to return the reddening vectors at the provided
        `av` and `rv`. Default is `False`.

    return_drvec : bool, optional
        Whether to return the differential reddening vectors at the provided
        `av` and `rv`. Default is `False`.

    Returns
    -------
    seds : `~numpy.ndarray` of shape `(Nmodels, Nbands)`
        Reddened SEDs.

    rvecs : `~numpy.ndarray` of shape `(Nmodels, Nbands)`, optional
        Reddening vectors.

    drvecs : `~numpy.ndarray` of shape `(Nmodels, Nbands)`, optional
        Differential reddening vectors.

    """

    Nmodels, Nbands, Ncoef = mag_coeffs.shape
    if av is None:
        av = np.zeros(Nmodels)
    elif isinstance(av, (int, float)):
        av = np.full(Nmodels, av)
    if rv is None:
        rv = np.full(Nmodels, 3.3)
    elif isinstance(rv, (int, float)):
        rv = np.full(Nmodels, rv)

    # Turn provided Av values into polynomial features.
    mags = mag_coeffs[:, :, 0]
    r0 = mag_coeffs[:, :, 1]
    dr = mag_coeffs[:, :, 2]

    # Compute SEDs.
    drvecs = np.array(dr)
    rvecs = r0 + rv[:, None] * drvecs
    seds = mags + av[:, None] * rvecs

    # Convert to flux.
    if return_flux:
        seds = 10**(-0.4 * seds)
        if return_rvec:
            rvecs *= -0.4 * np.log(10.) * seds
        if return_drvec:
            drvecs *= -0.4 * np.log(10.) * seds

    if return_rvec and return_drvec:
        return seds, rvecs, drvecs
    elif return_rvec:
        return seds, rvecs
    elif return_drvec:
        return seds, drvecs
    else:
        return seds